In [128]:
import tensorflow as tf
import keras
from keras.layers import Dense                                                                   
from keras.layers import Conv2D, AveragePooling2D, Dropout, Flatten      
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## Data Preprocessing

In [129]:
batch_size = 128                                                                                 
num_classes = 10                                                                                 
epochs = 12                                                                                      
img_rows, img_cols = 28, 28                                                                      

fashion_mnist = keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

train_images = train_images.reshape(train_images.shape[0], img_rows, img_cols, 1)
test_images = test_images.reshape(test_images.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

train_images = train_images.astype('float32')
test_images = test_images.astype('float32')
train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = np.pad(train_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')
test_images = np.pad(test_images, ((0,0),(2,2),(2,2),(0,0)), 'constant')

train_vec_labels = keras.utils.to_categorical(train_labels, total_classes)
test_vec_labels = keras.utils.to_categorical(test_labels, total_classes)

## Display Data

In [130]:
print(train_vec_labels[50])

[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


## Net Architectures

In [136]:
def LeNet5():    
    model = keras.Sequential()

    model.add(Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), activation='relu', input_shape=(32,32,1)))
    model.add(AveragePooling2D())

    model.add(Conv2D(filters=6, kernel_size=(5, 5), padding='VALID', activation='relu'))
    model.add(AveragePooling2D())

    model.add(Flatten())

    model.add(Dense(units=120, activation='relu'))

    model.add(Dense(units=84, activation='relu'))

    model.add(Dense(units=10, activation='softmax'))
    
    return model

## Compile and Train Network

In [135]:
model = LeNet5()
model.summary()


# sgd = keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['accuracy'])

model.fit(train_images, train_vec_labels,
          batch_size=batch_size,
          epochs=epochs,
          verbose=True,
          validation_data=(test_images, test_vec_labels))

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_42 (Conv2D)           (None, 28, 28, 64)        1664      
_________________________________________________________________
average_pooling2d_35 (Averag (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 10, 10, 32)        51232     
_________________________________________________________________
average_pooling2d_36 (Averag (None, 5, 5, 32)          0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 800)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 120)               96120     
_________________________________________________________________
dense_53 (Dense)             (None, 84)              

## Evaluate Model

In [133]:
eval_loss, eval_accuracy = model.evaluate(test_images, test_vec_labels, verbose=False)
print("Model accuracy: %.2f" % eval_accuracy)

Model accuracy: 0.29
